### modifyParameters
- read in CLM parameter files and creates 'new' pfts
- here copy C3 arctic grass (#12) for pft 13 and 14
- modify particular parameters for NWT simulations
- #12 = dry meadow , hh_col3
- #13 = moist meadow, hh_col1
- #14 = wet meadow, hh_col2 

Will Wieder Nov 2021

In [1]:
import xarray as xr
import pandas as pd
import os
import netCDF4
import numpy as np

### Parameter Files already modified to have 
- 10 day onset 
- 12h critDayLenth for offset, [peviously was using 13 hr file]


In [2]:
pathin = '/glade/work/wwieder/modified_inputs/NEON/'
basefile = pathin + 'ctsm51_params.c210803.NIWO_critDay12h_fastOnset.nc'
p = xr.open_dataset(basefile) 
#p

In [3]:
#p

In [4]:
mypft = 'c3_arctic_grass'
ix    = np.array([mypft in str(n) for n in p.pftname.values])  #index vector for mypft
#param = 'leaf_long'
param = 'medlynslope'
m = p[param].values
print(m[0:14])
print(m[ix])
p.pftname.values#[12]#[ix][0]


[0.         2.3499999  2.3499999  2.3499999  4.11999989 4.11999989
 4.44999981 4.44999981 4.44999981 4.69999981 4.69999981 4.69999981
 2.22000003 5.25      ]
[2.22000003]


array([b'not_vegetated                           ',
       b'needleleaf_evergreen_temperate_tree     ',
       b'needleleaf_evergreen_boreal_tree        ',
       b'needleleaf_deciduous_boreal_tree        ',
       b'broadleaf_evergreen_tropical_tree       ',
       b'broadleaf_evergreen_temperate_tree      ',
       b'broadleaf_deciduous_tropical_tree       ',
       b'broadleaf_deciduous_temperate_tree      ',
       b'broadleaf_deciduous_boreal_tree         ',
       b'broadleaf_evergreen_shrub               ',
       b'broadleaf_deciduous_temperate_shrub     ',
       b'broadleaf_deciduous_boreal_shrub        ',
       b'c3_arctic_grass                         ',
       b'c3_non-arctic_grass                     ',
       b'c4_grass                                ',
       b'c3_crop                                 ',
       b'c3_irrigated                            ',
       b'temperate_corn                          ',
       b'irrigated_temperate_corn                ',
       b'spr

In [5]:
# Frist overwrite pft 13-79 with pft specific values for C3 Arctic grass
p2 = p.copy(deep=True)
for var_name, values in p.items():
    if p2[var_name].size == 79 :
        print(var_name)
        p2[var_name].values[13:]=p2[var_name].values[12]

    # loop over phs parameters too
    if p2[var_name].size == 316 :
        print('phs '+var_name)
        for i in range(0,4):
            p2[var_name].values[i][13:]=p2[var_name].values[i][12]

    # loop over rootprof_beta parameters too
    if p2[var_name].size == 158 :
        print('rootprof '+var_name)
        for i in range(0,2):
            p2[var_name].values[i][13:]=p2[var_name].values[i][12]


FUN_fracfixers
a_fix
akc_active
akn_active
aleaff
allconsl
allconss
arootf
arooti
astemf
b_fix
baset
bfact
biofuel_harvfrac
c3psn
c_fix
cc_dstem
cc_leaf
cc_lstem
cc_other
phs ck
croot_stem
crop
deadwdcn
declfact
displar
dleaf
dsladlai
ekc_active
ekn_active
evergreen
fcur
fcurdv
fd_pft
ffrootcn
fleafcn
fleafi
flivewd
flnr
fm_droot
fm_dstem
fm_leaf
fm_lroot
fm_lstem
fm_other
fm_root
fnitr
fr_fcel
fr_flab
fr_flig
froot_leaf
frootcn
frootcn_max
frootcn_min
fsr_pft
fstemcn
fun_cn_flex_a
fun_cn_flex_b
fun_cn_flex_c
gddmin
graincn
grnfill
grperc
grpnow
hybgdd
i_flnr
i_vcad
irrigated
kc_nonmyc
phs kmax
kn_nonmyc
kr_resorb
krmax
laimx
leaf_long
leafcn
leafcn_max
leafcn_min
lf_fcel
lf_flab
lf_flig
lfemerg
lflitcn
livewdcn
livewdcn_max
livewdcn_min
lmr_intercept_atkin
manunitro
max_NH_planting_date
max_SH_planting_date
mbbopt
medlynintercept
medlynslope
mergetoclmpft
min_NH_planting_date
min_SH_planting_date
min_planting_temp
mxmat
mxtmp
pconv
perecm
pftnum
pftpar20
pftpar28
pftpar29
pftpar30
pft

In [6]:
# Check to make sure it worked
var_name = 'lmr_intercept_atkin'
#var_name = 'lnc'
#var_name = 'medlynslope'

var_name = 'kn_nonmyc'

p2[var_name].isel(pft=slice(0,15))

<xarray.DataArray 'kn_nonmyc' (pft: 15)>
array([0.    , 0.012 , 0.012 , 0.0012, 0.012 , 0.012 , 0.0012, 0.012 , 0.012 ,
       0.012 , 0.012 , 0.012 , 0.012 , 0.012 , 0.012 ])
Coordinates:
    pftname  (pft) |S40 b'not_vegetated                           ' ... b'c4_...
Dimensions without coordinates: pft
Attributes:
    long_name:  Constant relating soil layer Nitrogen content to non-mycorrhi...
    units:      gC/m2

---------------
#### parameter changes affecting all pfts
this won't place nicely with shrub simulations...

In [7]:
# Some additional parameter modifications for all simulations
p2['jmaxb0'].values = 0.025    #0.025 for Mod3 & Mod7-9, 0.02 for Mod6, 10, 11
# default 0.0331, 
# mods1, 2, 6, 12 = 0.02
# mods3, 7 = 0.025

#### Make specific changes for each pft
- 12 = Dry meadow
- 13 = Moist meadow
- 14 = Met meadow

- 11 = broadleaf_deciduous_boreal_shrub  

In [8]:
# For mod6, based on Katya's results from EDI
p2['FUN_fracfixers'].values[12:15] = [0.05 ,0.05  ,0.05]  # previouly used 0.05 for all 
                            # mod2, tried 0.25 for moist meadow with little effect
p2['slatop'].values[12:15]         = [0.015,0.0215,0.029] # previously used 0.018 from dry meadow, and no differences for wet meadow
p2['leafcn'].values[12:15]         = [18.5 ,19.8  ,17.7]  # prevously moist = wet meadow
p2['froot_leaf'].values[12:15]     = [2.5  ,2.0   ,2.0]   # prevously used 2.0 for dry meadow

# added for mod8, increasing LMR intercept for WM
p2['lmr_intercept_atkin'].values[12:15] = [2.1956,2.1956,2.4] #default 2.1956
# Also correct MM leafcn
p2['leafcn'].values[12:15]  = [18.5 ,19.6  ,17.7] 

# For mod9, increase fun uptake costs 2x
# for mod10, increase FUN updake costs 5x
p2['kn_nonmyc'].values[14]  = p2['kn_nonmyc'].values[14] * 5
p2['akn_active'].values[14]  = p2['akn_active'].values[14] * 5
p2['ekn_active'].values[14]  = p2['ekn_active'].values[14] * 5

# for mod10, decrease belowground C allocation for DM and MM
p2['froot_leaf'].values[12:15]     = [2.0  ,1.5   ,2.0]   # prevously used 2.0 for dry meadow

# for mod11, decrease belowground C allocation for DM and MM
p2['froot_leaf'].values[12:15]     = [2.0  ,1.5   ,1.5]   # prevously used 2.0 for dry meadow


In [15]:
# for parameter-11B increase belowground allocation
p2['froot_leaf'].values[11] = 2.5  #default 1.5

In [16]:
p2['froot_leaf']
# for mod12, reduce lmrha to reduce LAI?
#print(p2['lmrhd'])
#p2['lmrha']

#p2['lmrha'].values  = p2['lmrha'].values * 0.5


<xarray.DataArray 'froot_leaf' (pft: 79)>
array([0.  , 1.5 , 1.5 , 0.75, 1.5 , 1.5 , 1.5 , 1.5 , 0.75, 1.5 , 1.5 , 2.5 ,
       2.  , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 ,
       1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 ,
       1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 ,
       1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 ,
       1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 ,
       1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 , 1.5 ])
Coordinates:
    pftname  (pft) |S40 b'not_vegetated                           ' ... b'irr...
Dimensions without coordinates: pft
Attributes:
    long_name:  Allocation parameter: new fine root C per new leaf C
    units:      gC/gC

In [17]:
'''
p2['FUN_fracfixers'].values[12:15] = [0.05 ,0.05  ,0.05]  # previouly used 0.05 for all 
                            # mod2, tried 0.25 for moist meadow with little effect
p2['slatop'].values[12:15]         = [0.0175,0.023,0.026] # previously used 0.018 from dry meadow, and no differences for wet meadow
p2['leafcn'].values[12:15]         = [21.5 ,19.5  ,17.5]  # prevously moist = wet meadow
p2['froot_leaf'].values[12:15]     = [2.5  ,1.5   ,1.5]   # prevously used 2.0 for dry meadow
'''

"\np2['FUN_fracfixers'].values[12:15] = [0.05 ,0.05  ,0.05]  # previouly used 0.05 for all \n                            # mod2, tried 0.25 for moist meadow with little effect\np2['slatop'].values[12:15]         = [0.0175,0.023,0.026] # previously used 0.018 from dry meadow, and no differences for wet meadow\np2['leafcn'].values[12:15]         = [21.5 ,19.5  ,17.5]  # prevously moist = wet meadow\np2['froot_leaf'].values[12:15]     = [2.5  ,1.5   ,1.5]   # prevously used 2.0 for dry meadow\n"

In [18]:
p2['kmax']

<xarray.DataArray 'kmax' (segment: 4, pft: 79)>
array([[0.000000e+00, 1.602140e-08, 2.234574e-08, ..., 2.414841e-09,
        2.414841e-09, 2.414841e-09],
       [0.000000e+00, 1.602140e-08, 2.234574e-08, ..., 2.414841e-09,
        2.414841e-09, 2.414841e-09],
       [0.000000e+00, 1.602140e-08, 2.234574e-08, ..., 2.414841e-09,
        2.414841e-09, 2.414841e-09],
       [0.000000e+00, 1.602140e-08, 2.234574e-08, ..., 2.414841e-09,
        2.414841e-09, 2.414841e-09]])
Coordinates:
    pftname  (pft) |S40 b'not_vegetated                           ' ... b'irr...
  * segment  (segment) |S40 b'sunlit                                  ' ... b...
Dimensions without coordinates: pft
Attributes:
    units:      mm h2o (transpired)/mm h2o (water potential gradient)/sec
    long_name:  plant segment max conductance

In [19]:
# reduce kmax for dry meadow only
for i in range(0,4):
    p2['kmax'].values[i][12] = 2.3e-10
    if i == 0:
        print(p2['kmax'].values[i][12])
        print(p2['kmax'].values[i][13])

2.3e-10
2.4148408223121123e-09


### now write out a .nc file

In [20]:
p2.to_netcdf(pathin + 'ctsm51_params.c210803.NIWO_critDay12h_fastOnset_c3ArcticMods11b.nc')

In [21]:
pathin

'/glade/work/wwieder/modified_inputs/NEON/'

In [35]:
p2.fff

<xarray.DataArray 'fff' ()>
array(0.5)
Attributes:
    long_name:  Decay factor for fractional saturated area
    units:      1/m